In [3]:
import os
import ipywidgets as widgets
from IPython.display import display
import threading
import time
from PIL import Image, ImageDraw
import io
import uuid

import numpy as np
import cv2

In [4]:
os.makedirs('../dataset/relabel', exist_ok=True)

In [5]:
# HSV閾値のためのスライダーを作成
low_h_slider = widgets.IntSlider(min=0, max=180, value=90, description='Low H')
high_h_slider = widgets.IntSlider(min=0, max=180, value=100, description='High H')
low_s_slider = widgets.IntSlider(min=0, max=255, value=140, description='Low S')
high_s_slider = widgets.IntSlider(min=0, max=255, value=255, description='High S')
low_v_slider = widgets.IntSlider(min=0, max=255, value=0, description='Low V')
high_v_slider = widgets.IntSlider(min=0, max=255, value=255, description='High V')


def parse_filename(filename):
    """ ファイル名からx, y, rを抽出する。特定の形式に一致しない場合は(0, 0, 0)を返す """
    parts = filename.split('_')
    if len(parts) >= 4:
        try:
            x, y, r = map(int, parts[1:4])
            return x, y, r
        except ValueError:
            # 数値変換に失敗した場合もデフォルト値を返す
            return 0, 0, 0
    return 0, 0, 0

def draw_circle_on_image(image, x, y, r):
    """ 画像に円を描画する """
    image = cv2.circle(image, (x, y), r, (0, 255, 0), 2)
    return image
    
def apply_hsv_threshold(image):
    # low_h, high_h = 90, 100  # Example range for yellow hue
    # low_s, high_s = 140, 255 # Example range for saturation
    # low_v, high_v = 0, 255 # Example range for value

    low_h, high_h = low_h_slider.value, high_h_slider.value
    low_s, high_s = low_s_slider.value, high_s_slider.value
    low_v, high_v = low_v_slider.value, high_v_slider.value
    

    # Convert the image from RGB to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    # Define the lower and upper bounds of the HSV threshold
    lower_bound = np.array([low_h, low_s, low_v])
    upper_bound = np.array([high_h, high_s, high_v])
    
    # Create a mask where pixels within the threshold are white, and others are black
    mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    
    # Create an all black image
    black_image = np.zeros_like(image)
    
    # Copy the pixels from the original image where the mask is white
    result_image = np.where(mask[:, :, None] == 255, image, black_image)
    
    return result_image

In [6]:
def display_images_with_mask(image_directory, interval=0.5):
    # 画像ファイルのリストを取得
    images = [f for f in os.listdir(image_directory) if f.endswith(('.png', '.jpg', '.jpeg', '.gif'))]
    image_path = [os.path.join(image_directory, image) for image in images]
    total_images = len(images)

    is_playing = False
    current_image = 0
    # 画像とファイル名、インデックスを表示するためのウィジェットを作成
    image_widget = widgets.Image(
        value=open(image_path[0], 'rb').read() if images else None,
        format='jpg',
        width=224,
        height=224,
    )
    mask_image_widget = widgets.Image(
        format='jpg',
        width=224,
        height=224,
    )

    filename_label = widgets.Label(value=f"{images[0]} - 1/{total_images}" if images else "No images available")

    x, y, r = parse_filename(images[current_image])
    x_slider = widgets.IntSlider(min=0, max=224, value=x, description='X')
    y_slider = widgets.IntSlider(min=0, max=224, value=y, description='Y')
    r_slider = widgets.IntSlider(min=0, max=112, value=r, description='R')


    def reload_image(b):
        update_image()

    # 再描写ボタンの作成
    reload_button = widgets.Button(description="Reload")
    reload_button.on_click(reload_image)
    
    # 画像を更新する関数
    def update_image():
        nonlocal current_image
        if images:
            # original_image = Image.open(image_path[current_image])
            original_image = cv2.imread(image_path[current_image])
            mask_image = apply_hsv_threshold(original_image)
            
            
            x, y, r = x_slider.value, y_slider.value, r_slider.value
            image_with_circle = draw_circle_on_image(mask_image, x, y, r) if x is not None else open(image_path[current_image], 'rb').read()
            
            _, encoded_image = cv2.imencode('.jpg', image_with_circle)
            mask_image_widget.value = encoded_image.tobytes()
            image_widget.value = open(image_path[current_image], 'rb').read()

            
            filename_label.value = f"{images[current_image]} - {current_image + 1}/{total_images}"

    def save_image(b):
        nonlocal current_image
        if images:
            # 現在のスライダーの座標を取得
            x, y, r = x_slider.value, y_slider.value, r_slider.value

            # 新しいファイル名を生成
            base_name, ext = os.path.splitext(images[current_image])
            parts = base_name.split('_')
            if len(parts) >= 2:
                try:
                    base = parts[0] + 're'
                except ValueError:
                    base = 're'
            else:
                base = 're'

        # UUIDを追加
        unique_id = uuid.uuid4()
        new_filename = f"{base}_{x}_{y}_{r}_{unique_id}{ext}"

        # 画像を読み込み、保存
        image = cv2.imread(image_path[current_image])
        output_directory = '../dataset/relabel/'
        cv2.imwrite(os.path.join(output_directory, new_filename), image)
        
    def save_special_image(b):
        nonlocal current_image
        if images:
            # 特別なパラメータとUUIDを設定
            special_params = '-1_-1_0'
            unique_id = uuid.uuid4()
            base_name, ext = os.path.splitext(images[current_image])
            new_filename = f"re_{special_params}_{unique_id}{ext}"

            # 画像を読み込み、保存
            image = cv2.imread(image_path[current_image])
            output_directory = '../dataset/relabel/'
            cv2.imwrite(os.path.join(output_directory, new_filename), image)




    def on_back_clicked(b):
        nonlocal current_image
        if images:
            if current_image > 0:
                current_image -= 1
            else:
                current_image = total_images - 1
            update_image()

    def on_forward_clicked(b):
        nonlocal current_image
        if images:
            current_image += 1
            if current_image >= total_images:
                current_image = 0
            update_image()
    

    # スライダーの変更イベントハンドラ
    def on_slider_change(change):
        update_image()

    x_slider.observe(on_slider_change, names='value')
    y_slider.observe(on_slider_change, names='value')
    r_slider.observe(on_slider_change, names='value')

    # 再生・停止・戻る・削除ボタンの作成
    # 再生・停止・戻る・削除ボタンの作成
    
    back_button = widgets.Button(description="Back")
    forward_button = widgets.Button(description="Forward")
    forward_button.on_click(on_forward_clicked)
    back_button.on_click(on_back_clicked)
    # 保存ボタンの作成
    save_button = widgets.Button(description="Save")
    save_button.on_click(save_image)
    
    # 特別な保存ボタンの作成
    special_save_button = widgets.Button(description="Special Save")
    special_save_button.on_click(save_special_image)

    
    # ウィジェットを表示
    pos_sliders = widgets.VBox([x_slider, y_slider, r_slider])
    mask_sliders = widgets.VBox([low_h_slider, high_h_slider, low_s_slider, high_s_slider, low_v_slider, high_v_slider])


    image_view = widgets.HBox([image_widget, mask_image_widget])
    button_view = widgets.HBox([forward_button,back_button,  reload_button,save_button,special_save_button])
    sliders = widgets.HBox([mask_sliders,pos_sliders])

    total_widget = widgets.VBox([sliders, image_view, button_view])
    display(total_widget)
    display(filename_label)





In [7]:
display_images_with_mask('../dataset/label/', interval=0.5)

Label(value='9_162_183_25_b7b0dfeb161b40fb9016ffdac001005f.jpg - 1/30')